# IMPORTS

In [10]:
import re
import requests
import sqlite3

import pandas as pd
import numpy  as np

from datetime   import datetime
from bs4        import BeautifulSoup
from sqlalchemy import create_engine

## Helper functions

# DATA EXTRATION

In [2]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0'}

page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

## URL for all products

In [3]:
# getting number of products
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')
total_item

# calculating number of pages
page_number = np.ceil(int(total_item)/36)
page_number

# generating url
url02 = url + '?page-size=' + str(int(page_number)*36)
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [4]:
page = requests.get(url02, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

## Products details

In [5]:
products = soup.find('ul', class_='products-listing small')

product_list = products.find_all('article', class_='hm-product-item')

### Product ID


In [6]:
# product id
product_id = [p.get('data-articlecode') for p in product_list]

### Product Category

In [7]:
# product category
product_category = [p.get('data-category') for p in product_list]

### Product Name

In [8]:
# product name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]

### Product Price

In [9]:
# price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

### Creating Dataset

In [10]:
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [11]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1100162002,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54
1,1100162003,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54
2,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-03 10:00:54
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-03 10:00:54
4,0690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-03 10:00:54


### Product Color and Composition

In [12]:
# empty dataframe for append
df_color = pd.DataFrame()
df_composition = pd.DataFrame()

# iteration for each id product
for code in data['product_id']:
    url02 = 'https://www2.hm.com/en_us/productpage.' + str(code) + '.html'

    page = requests.get(url02, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Product Color
    color_name = soup.find('a', class_='filter-option miniature active').get('data-color')

    # product id
    product_code = soup.find('a', class_='filter-option miniature active').get('data-articlecode')

    aux1 = pd.DataFrame({'product_id': product_code, 'color_name': color_name}, index=[0])
    df_color = pd.concat([df_color, aux1])

    # Product Composition 
    product_composition_list = soup.find_all('div', class_='details-attributes-list-item')

    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]
    
    composition = pd.DataFrame(product_composition).T

    # rename dataframe
    composition.columns = composition.iloc[0]

    # delete first row
    composition['Art. No.'] = composition['Art. No.'].fillna(method='ffill')
    composition = composition.iloc[1:]

    composition_aux = composition.fillna('').groupby(['Art. No.'], as_index=False).sum()

    df_composition = pd.concat([df_composition, composition_aux], axis=0)
    
df_composition = df_composition[['Art. No.', 'Fit', 'Size', 'Composition', 'Additional material information']]
df_composition.rename(columns={'Art. No.': 'product_id', 'Fit': 'fit', 'Size': 'size', 'Composition':
                               'composition', 'Additional material information': 'additional_material'},
                               inplace=True)

# merging the dataframes
df_details = pd.merge(df_color, df_composition, how='left', on='product_id')

data = pd.merge(data, df_details, how='left', on='product_id')
data

,product_id,product_category,product_name,product_price,scrapy_datetime,color_name,fit,size,composition,additional_material
0,1100162002,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54,Denim blue,Regular fit,Inner leg: Length: 79.5 cm (Size 33/32),"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
1,1100162003,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54,Denim gray,Regular fit,Inner leg: Length: 79.5 cm (Size 33/32),"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
2,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-03 10:00:54,Black,Skinny fit,NaN,"Shell: Cotton 99%, Spandex 1%Pocket lining: Co...",Recycled cotton 20%
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-03 10:00:54,Black,Slim fit,NaN,"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
4,0690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-03 10:00:54,Denim blue,Skinny fit,NaN,"Shell: Cotton 98%, Spandex 2%Pocket lining: Po...",NaN
...,...,...,...,...,...,...,...,...,...,...
87,1025726003,men_jeans_relaxed,Relaxed Jeans,$ 39.99,2022-11-03 10:00:54,Graphite gray,Relaxed fit,Inner leg: Length: 81.0 cm (Size 33/32)Waist: ...,Shell: Cotton 100%Pocket lining: Polyester 65%...,NaN
88,1027852007,men_jeans_joggers,Relaxed Denim Joggers,$ 29.99,2022-11-03 10:00:54,Denim blue,Relaxed fit,Inner leg: Length: 77.7 cm (Size L),Shell: Cotton 100%Pocket lining: Cotton 100%,NaN
89,0875105016,men_jeans_relaxed,Relaxed Jeans,$ 39.99,2022-11-03 10:00:54,Denim blue,Relaxed fit,Inner leg: Length: 89.5 cm (Size 33)Waist: Cir...,Shell: Cotton 100%Pocket lining: Polyester 65%...,NaN
90,0811993037,men_jeans_regular,Regular Jeans,$ 29.99,2022-11-03 10:00:54,Dark blue,Regular fit,NaN,"Cotton 99%, Spandex 1%",Recycled cotton 20%


## Saving the dataset

In [13]:
#data.to_csv('data_hm.csv', index=False)

# DATA MANIPULATION

In [2]:
data = pd.read_csv('data_hm.csv')

In [3]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,color_name,fit,size,composition,additional_material
0,1100162002,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54,Denim blue,Regular fit,Inner leg: Length: 79.5 cm (Size 33/32),"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
1,1100162003,men_jeans_regular,Essentials No 2: THE JEANS,$ 39.99,2022-11-03 10:00:54,Denim gray,Regular fit,Inner leg: Length: 79.5 cm (Size 33/32),"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
2,985159001,men_jeans_skinny,Skinny Jeans,$ 24.99,2022-11-03 10:00:54,Black,Skinny fit,NaN,"Shell: Cotton 99%, Spandex 1%Pocket lining: Co...",Recycled cotton 20%
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-03 10:00:54,Black,Slim fit,NaN,"Shell: Cotton 99%, Spandex 1%Pocket lining: Po...",NaN
4,690449067,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-03 10:00:54,Denim blue,Skinny fit,NaN,"Shell: Cotton 98%, Spandex 2%Pocket lining: Po...",NaN


In [4]:
# product_name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price
data['product_price'] = data['product_price'].apply(lambda x: x.replace ('$ ', '')).astype(float)

# scrapy_datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# color_name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').lower())

# Fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower())

In [5]:
# Composition - Shell
data['shell_composition'] = np.NaN

for index, line in data.iterrows():
    # since there is no pattern, different methods are implemented
    if re.match('Shell:.(.+%)\w', line['composition']):
        data.loc[index, 'shell_composition'] = re.match('Shell:.(.+%)\w', line['composition']).group(1)
    elif re.match('(.+)%\w', line['composition']):
        data.loc[index, 'shell_composition'] = re.match('(.+)%\w', line['composition']).group(1)
    else:
        data.loc[index, 'shell_composition'] = line['composition']

# Composition - Lining

for index, line in data.iterrows():
    if re.search('Pock.+: (.+)', line['composition']):
        data.loc[index, 'pocket_lining_composition'] = re.search('Pock.+: (.+)', line['composition']).group(1)

# size
data['size_number'] = np.NaN
data['leg_lenght'] = np.NaN
data['circumference'] = np.NaN

for index, line in data.iterrows():
    # look for lines with text for extraction
    if pd.notnull(line['size']):
        if re.search('Length: (.{1,4}) cm', line['size']):
            data.loc[index, 'leg_lenght'] = re.search('Length: (.{1,4}) cm', line['size']).group(1)
        if re.search('Circumference: (.{1,4}) cm', line['size']):
            data.loc[index, 'circumference'] = re.search('Circumference: (.{1,4}) cm', line['size']).group(1)

        if re.search('\(Size (.+)\)\w', line['size']):
            data.loc[index, 'size_number'] = re.search('\(Size (.+)\)\w', line['size']).group(1)
        elif re.search('\(Size (.+)\)', line['size']):
            data.loc[index, 'size_number'] = re.search('\(Size (.+)\)', line['size']).group(1)

In [6]:
# Spliting shell composition

# Cotton, Spandex, Polyester, Elastomultiester, Rayon, Lyocell
data['cotton'] = data['shell_composition'].apply(lambda x: int(re.search('Cotton (\d{1,3})', x).group(1))/100
                                                 if re.search('Cotton (\d{1,3})', x) else np.NaN)

data['spandex'] = data['shell_composition'].apply(lambda x: int(re.search('Spandex (\d{1,3})',x).group(1))/100
                                                  if re.search('Spandex (\d{1,3})', x) else np.NaN)

data['polyester'] = data['shell_composition'].apply(lambda x:
                                                    int(re.search('Polyester (\d{1,3})',x).group(1))/100
                                                    if re.search('Polyester (\d{1,3})', x) else np.NaN)

data['elastomultiester'] = data['shell_composition'].apply(lambda x:
                                                           int(re.search('Elastomultiester (\d{1,3})',
                                                                         x).group(1))/100 
                                                           if re.search('Elastomultiester (\d{1,3})', x) else
                                                           np.NaN)

data['rayon'] = data['shell_composition'].apply(lambda x: int(re.search('Rayon (\d{1,3})', x).group(1))/100
                                                if re.search('Rayon (\d{1,3})', x) else np.NaN)

data['lyocell'] = data['shell_composition'].apply(lambda x: int(re.search('Lyocell (\d{1,3})',x).group(1))/100
                                                  if re.search('Lyocell (\d{1,3})', x) else np.NaN)

data = data.drop(['size', 'composition', 'shell_composition'], axis=1)

In [7]:
for index, line in data.iterrows():
    # check for null values
    if pd.notnull(line['pocket_lining_composition']):
        if re.search('Cotton (\d{1,3})', line['pocket_lining_composition']):
            data.loc[index, 'cotton_pocket'] = int(re.search('Cotton (\d{1,3})',
                                                         line['pocket_lining_composition']).group(1))/100
        
        if re.search('Polyester (\d{1,3})', line['pocket_lining_composition']):
            data.loc[index, 'polyester_pocket'] = int(re.search('Polyester (\d{1,3})',
                                                         line['pocket_lining_composition']).group(1))/100
    if pd.notnull(line['additional_material']):
        if re.search('cotton (\d{1,3})', line['additional_material']):
            data.loc[index, 'recycled_cotton'] = int(re.search('cotton (\d{1,3})',
                                                        line['additional_material']).group(1))/100
        
        if re.search('polyester (\d{1,3})', line['additional_material']):
            data.loc[index, 'recycled_polyester'] = int(re.search('cotton (\d{1,3})',
                                                        line['additional_material']).group(1))/100
            
data = data.drop(['pocket_lining_composition', 'additional_material'], axis=1)

In [8]:
data

,product_id,product_category,product_name,product_price,scrapy_datetime,color_name,fit,size_number,leg_lenght,circumference,cotton,spandex,polyester,elastomultiester,rayon,lyocell,cotton_pocket,polyester_pocket,recycled_cotton,recycled_polyester
0,1100162002,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-11-03 10:00:54,denim_blue,regular_fit,33/32,79.5,NaN,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
1,1100162003,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-11-03 10:00:54,denim_gray,regular_fit,33/32,79.5,NaN,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
2,985159001,men_jeans_skinny,skinny_jeans,24.99,2022-11-03 10:00:54,black,skinny_fit,NaN,NaN,NaN,0.99,0.01,NaN,NaN,NaN,NaN,1.00,NaN,0.2,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-11-03 10:00:54,black,slim_fit,NaN,NaN,NaN,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
4,690449067,men_jeans_ripped,skinny_jeans,39.99,2022-11-03 10:00:54,denim_blue,skinny_fit,NaN,NaN,NaN,0.98,0.02,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1025726003,men_jeans_relaxed,relaxed_jeans,39.99,2022-11-03 10:00:54,graphite_gray,relaxed_fit,33/32,81.0,90.5,1.00,NaN,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
88,1027852007,men_jeans_joggers,relaxed_denim_joggers,29.99,2022-11-03 10:00:54,denim_blue,relaxed_fit,L,77.7,NaN,1.00,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN
89,875105016,men_jeans_relaxed,relaxed_jeans,39.99,2022-11-03 10:00:54,denim_blue,relaxed_fit,33,89.5,90.5,1.00,NaN,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
90,811993037,men_jeans_regular,regular_jeans,29.99,2022-11-03 10:00:54,dark_blue,regular_fit,NaN,NaN,NaN,0.99,0.01,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN


# Inserting data to DataBase

In [12]:
data.dtypes

product_id                     int64
product_category              object
product_name                  object
product_price                float64
scrapy_datetime       datetime64[ns]
color_name                    object
fit                           object
size_number                   object
leg_lenght                    object
circumference                 object
cotton                       float64
spandex                      float64
polyester                    float64
elastomultiester             float64
rayon                        float64
lyocell                      float64
cotton_pocket                float64
polyester_pocket             float64
recycled_cotton              float64
recycled_polyester           float64
dtype: object

In [13]:
# query for the table creation
query_showroom_schema = """
    CREATE TABLE showroom (
        product_id         INTEGER,
        product_category   TEXT,
        product_name       TEXT,
        product_price      REAL,
        scrapy_datetime    TEXT,
        color_name         TEXT,
        fit                TEXT,
        size_number        TEXT,
        leg_lenght         TEXT, 
        circumference      TEXT,
        cotton             REAL,
        spandex            REAL,
        polyester          REAL,
        elastomultiester   REAL,  
        rayon              REAL,
        lyocell            REAL,
        cotton_pocket      REAL,
        polyester_pocket   REAL,   
        recycled_cotton    REAL,
        recycled_polyester REAL
    )
"""

## Creating DataBase

In [14]:
conn = sqlite3.connect('hm_db.sqlite')

# plan the execution
cursor = conn.execute(query_showroom_schema)

conn.commit()
conn.close()

## Inserting data

In [15]:
# other way to connect
conn = create_engine('sqlite:///hm_db.sqlite', echo=False)

In [16]:
data.to_sql('showroom', con=conn, if_exists='append', index=False)

92

In [18]:
query = """
  SELECT * FROM showroom
"""

df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,color_name,fit,size_number,leg_lenght,circumference,cotton,spandex,polyester,elastomultiester,rayon,lyocell,cotton_pocket,polyester_pocket,recycled_cotton,recycled_polyester
0,1100162002,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-11-03 10:00:54.000000,denim_blue,regular_fit,33/32,79.5,None,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
1,1100162003,men_jeans_regular,essentials_no_2:_the_jeans,39.99,2022-11-03 10:00:54.000000,denim_gray,regular_fit,33/32,79.5,None,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
2,985159001,men_jeans_skinny,skinny_jeans,24.99,2022-11-03 10:00:54.000000,black,skinny_fit,None,None,None,0.99,0.01,NaN,NaN,NaN,NaN,1.00,NaN,0.2,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-11-03 10:00:54.000000,black,slim_fit,None,None,None,0.99,0.01,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN
4,690449067,men_jeans_ripped,skinny_jeans,39.99,2022-11-03 10:00:54.000000,denim_blue,skinny_fit,None,None,None,0.98,0.02,NaN,NaN,NaN,NaN,0.35,0.65,NaN,NaN


## Commands

In [ ]:
# command - UPDATE
query = """
    UPDATE showroom
    SET product_category = 'tshirt'
    WHERE product_id = 1100162002
"""

# command - DROP TABLE
query = """
    DROP TABLE showroom
"""

# command - ALTER TABLE
query = """
    ALTER TABLE showroom
    RENAME TO showroom_two
"""

# command - CREATE INDEX
query = """
    CREATE INDEX idx_product_id
    ON showroom_two (product_id)
"""

conn = sqlite3.connect('hm_db.sqlite')

# plan the execution
cursor = conn.execute()

conn.commit()
conn.close()